In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!wget https://repo.anaconda.com/archive/Anaconda3-2018.12-Linux-x86_64.sh
!chmod +x Anaconda3-2018.12-Linux-x86_64.sh
!bash ./Anaconda3-2018.12-Linux-x86_64.sh -b -f -p /usr/local

--2019-03-19 07:08:38--  https://repo.anaconda.com/archive/Anaconda3-2018.12-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684237703 (653M) [application/x-sh]
Saving to: ‘Anaconda3-2018.12-Linux-x86_64.sh’

Anaconda3-2018.12-L 100%[===================>] 652.54M   120MB/s    in 5.2s    

2019-03-19 07:08:43 (125 MB/s) - ‘Anaconda3-2018.12-Linux-x86_64.sh’ saved [684237703/684237703]

PREFIX=/usr/local
reinstalling: python-3.7.1-h0371630_7 ...
Python 3.7.1
reinstalling: blas-1.0-mkl ...
reinstalling: ca-certificates-2018.03.07-0 ...
reinstalling: conda-env-2.6.0-1 ...
reinstalling: intel-openmp-2019.1-144 ...
reinstalling: libgcc-ng-8.2.0-hdf63c60_1 ...
reinstalling: libgfortran-ng-7.3.0-hdf63c60_0 ...
reinstalling: libstdcxx-ng-8.2.0-hdf63c60_1 ...
reinstalling: bzip2-1.0.6-h1

In [4]:
!conda create -n tensorflow1 pip python=3.5
!activate tensorflow1
!pip install --ignore-installed --upgrade tensorflow-gpu
!conda install -c anaconda protobuf
!pip install pillow
!pip install Cython
!pip install lxml
!pip install jupyter
!pip install matplotlib
!pip install pandas
!pip install opencv-python
!cd ..
!cd content
!cp gdrive/'My Drive'/tensorflow1.zip /content
!unzip tensorflow1.zip
!set PYTHONPATH=\content\tensorflow1\models;\content\tensorflow1\models\research;\content\tensorflow1\models\research\slim
!rm tensorflow1.zip
!cd tensorflow1/models/research
!protoc --python_out=. ./object_detection/protos/anchor_generator.proto ./object_detection/protos/argmax_matcher.proto ./object_detection/protos/bipartite_matcher.proto ./object_detection/protos/box_coder.proto ./object_detection/protos/box_predictor.proto ./object_detection/protos/eval.proto ./object_detection/protos/faster_rcnn.proto ./object_detection/protos/faster_rcnn_box_coder.proto ./object_detection/protos/grid_anchor_generator.proto ./object_detection/protos/hyperparams.proto ./object_detection/protos/image_resizer.proto ./object_detection/protos/input_reader.proto ./object_detection/protos/losses.proto ./object_detection/protos/matcher.proto ./object_detection/protos/mean_stddev_box_coder.proto ./object_detection/protos/model.proto ./object_detection/protos/optimizer.proto ./object_detection/protos/pipeline.proto ./object_detection/protos/post_processing.proto ./object_detection/protos/preprocessor.proto ./object_detection/protos/region_similarity_calculator.proto ./object_detection/protos/square_box_coder.proto ./object_detection/protos/ssd.proto ./object_detection/protos/ssd_anchor_generator.proto ./object_detection/protos/string_int_label_map.proto ./object_detection/protos/train.proto ./object_detection/protos/keypoint_box_coder.proto ./object_detection/protos/multiscale_anchor_generator.proto ./object_detection/protos/graph_rewriter.proto
!python setup.py build
!python setup.py install
!cd object_detection
!python sizeChecker.py --move
!python Object_detection_webcam.py

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.8

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local/envs/tensorflow1

  added / updated specs: 
    - pip
    - python=3.5


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h7b6447c_0         3.2 MB
    sqlite-3.27.2              |       h7b6447c_0         1.9 MB
    certifi-2018.8.24          |           py35_1         139 KB
    setuptools-40.2.0          |           py35_0         571 KB

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [5]:
cd tensorflow1/models/research/object_detection

/content/tensorflow1/models/research/object_detection


In [6]:
ls

anchor_generators/                         matchers/
box_coders/                                meta_architectures/
builders/                                  metrics/
CONTRIBUTING.md                            model_hparams.py
core/                                      model_lib.py
data/                                      model_lib_test.py
data_decoders/                             model_main.py
dataset_tools/                             models/
doc/                                       model_tpu_main.py
dockerfiles/                               Object_detection_image.py
eval_util.py                               object_detection_tutorial.ipynb
eval_util_test.py                          Object_detection_video.py
exporter.py                                Object_detection_webcam.py
exporter_test.py                           predictors/
export_inference_graph.py                  protos/
export_tflite_ssd_graph_lib.py             README.md
export_tflite_ssd_graph_lib_test.py        r

In [7]:
!python sizeChecker.py --move

[Ok] Directories exists
sizeChecker.py:45: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  xmin, ymin, xmax, ymax = box.getchildren()
[Ok] All bounding boxes are equal or larger than 32 :-)


In [8]:
!pip uninstall numpy
!pip uninstall numpy

Uninstalling numpy-1.16.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/site-packages/numpy-1.16.2.dist-info/*
    /usr/local/lib/python3.7/site-packages/numpy/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/site-packages/numpy/_import_tools.py
    /usr/local/lib/python3.7/site-packages/numpy/_mklinit.cpython-37m-x86_64-linux-gnu.so
    /usr/local/lib/python3.7/site-packages/numpy/add_newdocs.py
    /usr/local/lib/python3.7/site-packages/numpy/core/multiarray.cpython-37m-x86_64-linux-gnu.so
    /usr/local/lib/python3.7/site-packages/numpy/core/umath.cpython-37m-x86_64-linux-gnu.so
    /usr/local/lib/python3.7/site-packages/numpy/distutils/environment.py
    /usr/local/lib/python3.7/site-packages/numpy/distutils/site.cfg
    /usr/local/lib/python3.7/site-packages/numpy/fft/tests/test_fft.py
    /usr/local/lib/python3.7/site-packages/numpy/random_intel/__init__.py
    /usr/local/lib/p

In [9]:
!pip install numpy

  Using cached https://files.pythonhosted.org/packages/91/e7/6c780e612d245cca62bc3ba8e263038f7c144a96a54f877f3714a0e8427e/numpy-1.16.2-cp37-cp37m-manylinux1_x86_64.whl


In [10]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2019-03-19 07:18:41--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.204.136.9, 52.3.53.115, 52.45.248.161, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.204.136.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14910739 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.22M  17.9MB/s    in 0.8s    

2019-03-19 07:18:42 (17.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14910739/14910739]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
model_dir = 'training/'
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [13]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://8fcdd6b8.ngrok.io


In [14]:
!ls {model_dir}

labelmap.pbtxt


In [15]:
import os
import glob

pb_fname = '/content/tensorflow1/models/research/object_detection/inference_graph/frozen_inference_graph.pb'
label_map_pbtxt_fname = '/content/tensorflow1/models/research/object_detection/training/labelmap.pbtxt'
repo_dir_path = '/content/tensorflow1/models/research/object_detection/images/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "testing")

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

['/content/tensorflow1/models/research/object_detection/images/testing/ah7.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/raptors16.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/af1b7.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/mauve14.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/huspecies9.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/glob7.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/aj1bred18.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/italy7.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/pslr14.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/vansblue9.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/presto15.jpg', '/content/tensorflow1/models/research/object_detection/images/testing/vansb

In [17]:
%cd /content/tensorflow1/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

        
num_classes = 20
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

Output hidden; open in https://colab.research.google.com to view.